#Minicurso de visão computacional

##Pacotes necessários

In [ ]:
# Verificar a disponibilidade de GPUs
!nvidia-smi

Tue Jan 14 16:47:33 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Instalar a versão especificada da biblioteca Ultralytics para YOLOv8
!pip install ultralytics==8.0.196

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.1/631.1 kB 11.6 MB/s eta 0:00:00


In [ ]:
# Importação de bibliotecas necessárias
from ultralytics import YOLO  # Biblioteca principal para treinar e usar o modelo YOLOv8
import os  # Para manipulação de arquivos e diretórios
from IPython.display import Image, display  # Para exibir imagens no Jupyter Notebook
from IPython import display
import glob
display.clear_output()  # Limpar a saída para facilitar a visualização

In [ ]:
#!yolo mode=check

In [ ]:
# Instalar a biblioteca Roboflow para gerenciamento de conjuntos de dados
!pip install roboflow
from roboflow import Roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


## Baixar o conjunto de dados do projeto no Roboflow


In [ ]:
rf = Roboflow(api_key="15adwlz40t0T4gQbaM54")  # Substitua pela sua chave de API, se necessário
project = rf.workspace("roof-8rynq").project("detect-fault-in-roof")  # Nome do workspace e projeto no Roboflow
dataset = project.version(2).download("yolov8")  # Fazer download da versão 2 do conjunto de dados

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Detect-fault-in-roof-2 in yolov8:: 100%|██████████| 1227/1227 [00:00<00:00, 6253.14it/s]


## Treinar o modelo YOLOv8 com os dados baixados


In [ ]:
#os.environ['WANDB_MODE'] = 'disabled' #Desativar o Weights & Biases (W&B)

In [ ]:
!yolo task=detect mode=train model=yolov8n.pt data={dataset.location}/data.yaml epochs=100 imgsz=640
# - task=detect: Define a tarefa como detecção de objetos.
# - mode=train: Inicia o treinamento do modelo.
# - model=yolov8n.pt: Utiliza o modelo YOLOv8 nano (mais leve).
# - data={dataset.location}/data.yaml: Aponta para o arquivo YAML com as configurações do conjunto de dados.
# - epochs=100: Define o número de épocas.
# - imgsz=640: Define o tamanho das imagens.

### Exibir os resultados do treinamento


In [ ]:
Image(filename=f'/content/runs/detect/train/results.png', width=800)

In [ ]:
Image(filename=f'/content/runs/detect/train/PR_curve.png', width=580)

In [ ]:
Image(filename=f'/content/runs/detect/train/confusion_matrix.png', width=580)

## Avaliar o modelo treinado usando o conjunto de validação


In [ ]:
!yolo task=detect mode=val model=/content/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml
# - model=/content/runs/detect/train/weights/best.pt: Utiliza os melhores pesos do treinamento.
# - data={dataset.location}/data.yaml: Aponta para o conjunto de dados para validação.

### Exibir os resultados da validação


In [ ]:
Image(filename=f'/content/runs/detect/val/PR_curve.png', width=560)

In [ ]:
Image(filename=f'/content/runs/detect/val/confusion_matrix.png', width=560)

## Fazer inferência em novas imagens

In [ ]:
!yolo task=detect mode=predict model=/content/runs/detect/train/weights/best.pt  conf=0.5 iou=0.5 source='/content/Detect-fault-in-roof-2/test/images'
# - conf=0.5: Define o limiar de confiança para as detecções.
# - iou=0.5: Define o limiar de interseção sobre união.
# - source: Caminho para as imagens de teste.


### Visualizar as imagens com as predições


In [ ]:
for imagepath in glob.glob(f'/content/runs/detect/predict/*.jpg'):
      display(Image(filename=imagepath, height=500))
      print('\n')

## Carregar o modelo treinado para avaliação detalhada

In [ ]:
model = YOLO("/content/runs/detect/train/weights/best.pt")

In [ ]:
# Avaliar o modelo em diferentes divisões do conjunto de dados
results = model.val(data='/content/Detect-fault-in-roof-2/data.yaml', split='train', imgsz=640)  # Conjunto de treinamento
results = model.val(data='/content/Detect-fault-in-roof-2/data.yaml', split='test', imgsz=640)  # Conjunto de teste
results = model.val(data='/content/Detect-fault-in-roof-2/data.yaml', split='val', imgsz=640)  # Conjunto de validação